In [ ]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.simplefilter('ignore')

In [ ]:
import ipyvuetify as v
import vaex.jupyter.widgets as vw
import ipywidgets as widgets
import vaex
import vaex.jupyter
import numpy as np
import getpass

colors = {
    'primary': '#1976D2',
    'secondary': '#424242',
    'accent': '#82B1FF',
    'error': '#FF5252',
    'info': '#2196F3',
    'success': '#4CAF50',
    'warning': '#FFC107'
}

In [ ]:
# Uncomment to access the data via S3
import vaex
import getpass

profile_name = 'stsci' if getpass.getuser() == 'maartenbreddels' else 'default'

df = vaex.open(f's3://astrosurveydata/gaia_ps1_nochunk.hdf5?profile_name={profile_name}')
# df = df[:20_000_000]
topic = 'stars'
default_custom_selection = str(df.parallax/df.parallax_error > 3)
default_custom_selection

In [ ]:
# import vaex
# with open('token.txt') as f:
#     token = f.read().strip()
# df = vaex.open(f'ws://ec2-18-222-183-211.us-east-2.compute.amazonaws.com:9000/gaia_ps1_nochunk?token_trusted={token}')#[:10_000_000]

In [ ]:
df

In [ ]:

sky_x = df.ra
sky_y = df.dec
limits_sky_x = [0, 360]
limits_sky_y = [-90, 90]

color = df.phot_bp_mean_mag - df.phot_g_mean_mag
df['distance'] = 1/df.parallax
df['M_g'] = df.phot_g_mean_mag - 5 * np.log10(df.distance) - 10
magnitude = df.M_g
limits_color = [-0.5, 1.5]
limit_magnitude = [22, -10]

aux = df.phot_g_mean_mag
limits_aux = [5, 23]
shape_aux = 100

In [ ]:
# TODO: vaex cannot work with 'empty' selection, make selection that use all the data
df.select(df.phot_g_mean_mag > 10, name='visual')
df.select(default_custom_selection, name='custom')
df.select(df.ra > -1000, name='sky')
df.select('visual & custom & sky')

# since the default selection depepends on this, and vaex does not
# track the dependencies, we have to update the default selection manually
@df.signal_selection_changed.connect
def update_default_selection(df, selection_name):
    if selection_name in 'visual custom sky'.split():
        df.select('visual & custom & sky')


In [ ]:
counter_selection = vw.AnimatedCounter(value=1234567, postfix=f' {topic} selected')
counter_selection.value = df.count(selection=True).item()
counter_selection

In [ ]:
counts_aux = df.count(aux, binby=aux, limits=limits_aux, shape=shape_aux, selection=[True, None])
counts_aux

In [ ]:
df.bin_centers(aux, limits_aux, shape_aux)

In [ ]:
import bqplot.pyplot as plt
fig1d = plt.figure()
fig1d.fig_margin = {'bottom': 35, 'left': 60, 'right': 5, 'top': 5}
bar = plt.bar(df.bin_centers(aux, limits_aux, shape_aux), counts_aux, type='grouped', colors = [colors['warning'], colors['accent']])
bar.scales['y'].allow_padding = False

fig1d.axes[0].label_offset = '30px'
fig1d.axes[0].label = str(aux)
fig1d.axes[1].label_offset = '50px'
fig1d.axes[1].label = 'counts'

fig1d.axes[0].color = \
fig1d.axes[1].color = \
fig1d.axes[0].label_color = \
fig1d.axes[1].label_color = \
fig1d.axes[0].grid_color = \
fig1d.axes[1].grid_color = '#666'

# even handling
for axes in fig1d.axes:
    axes.grid_lines = 'none'
selector = plt.brush_int_selector()
# plt.show()
fig1d

In [ ]:
# listen to the selector, and modify the selections
def update_selection_visual(*ignore):
    if selector.selected is not None and len(selector.selected) == 2:
        xmin, xmax = selector.selected
        df.select((aux > xmin) & (aux < xmax), name='visual')
selector.observe(update_selection_visual, 'selected')

In [ ]:
counter_processed = vw.AnimatedCounter(value=1234567, postfix=f' {topic} processed')
counter_processed.value = len(df)
counter_processed

In [ ]:
progress_circular = vw.ProgressCircularNoAnimation(width=10, size=70, color=colors['accent'], text='')

import time
last_time = time.time()
@df.executor.signal_begin.connect
def progress_begin():
    progress_circular.hidden = False
@df.executor.signal_progress.connect
def update_progress(value):
    global last_time
    progress_circular.value = value*100
    number = int(value * len(df))
    current_time = time.time()
    if (current_time - last_time) > 0.2 or value in [0, 1]:
        counter_processed.value = number
        last_time = current_time
    return True
@df.executor.signal_end.connect
def progress_update():
    progress_circular.hidden = True
    

progress = v.Layout(children=[progress_circular, counter_processed], align_center=True)
progress

In [ ]:
progress_circular.hidden = True

In [ ]:
# maybe we should put this in vaex-core?
@vaex.jupyter.debounced(0.05)
def _execute():
    df.execute()

output = widgets.Output()

# The update is debounced, since it is expensive to calculate
@vaex.jupyter.debounced(0.5)
def _update_plot1d():
    with output:
        @vaex.delayed
        def do(grid1d, count):
            with output:
                bar.y = grid1d
                counter_selection.value = count.item()
        counts_aux = df.count(aux, binby=aux, limits=limits_aux, shape=shape_aux, selection=[True, None], delay=True)
        do(counts_aux, df.count(selection=True, delay=True))
        _execute()

# this will be called when a selection changes
@df.signal_selection_changed.connect
def _selection_changed(df, selection_name):
    if selection_name == 'default':
        _update_plot1d()

In [ ]:
output

In [ ]:
plot_sky = df.plot_widget(sky_x, sky_y, f='log', selection=[False, 'sky'], limits=[limits_sky_x, limits_sky_y], selection_callback=update_default_selection)
plot_sky.widget.title = f'All {topic}'
c = plot_sky.widget.components['main-widget']
c.layout.padding = '60px 0px 0px 0px'

In [ ]:
plot_hrd = df.plot_widget(color, magnitude, f='log', selection=True, limits=[limits_color, limit_magnitude])
plot_hrd.widget.title = f'Selected {topic}'
c = plot_hrd.widget.components['main-widget']
c.layout.padding = '60px 0px 0px 0px'

In [ ]:
for fig in [fig1d, plot_sky.backend.figure, plot_hrd.backend.figure]:
    fig.layout.width = 'auto'
    fig.layout.width = 'auto'
    fig.layout.height = 'auto'
    fig.layout.min_height = '400px' # so it still shows nicely in the notebook
    fig.layout.min_width = 'auto'
    fig.layout.min_width = 'auto'

In [ ]:
expression_area = vw.ExpressionTextArea(df=df)
expression_area

In [ ]:
widget_selection_text_area = vw.ExpressionSelectionTextArea(df=df, selection_name='custom', v_model=default_custom_selection)
widget_selection_text_area

In [ ]:
widget_selection = vw.SelectionEditor(df=df, input=widget_selection_text_area)
widget_selection

In [ ]:
editor = vw.VirtualColumnEditor(df=df)
editor

In [ ]:
column_list = vw.ColumnList(df=df)
column_list

In [ ]:
import traitlets
class Action(v.VuetifyTemplate):
    value = traitlets.Bool(False).tag(sync=True)
    backend = traitlets.Any()
    items = traitlets.Any().tag(sync=True)
    @traitlets.default('template')
    def _template(self):
        return """
        <v-speed-dial
        style="bottom: 80px"
        absolute
      v-model="value"
      bottom
      right
      direction="top"
      transition="slide-y-reverse-transition"
    >
      <template v-slot:activator>
        <v-btn
          v-model="value"
          small
          fab
        >
          <v-icon v-if="value">expand_more</v-icon>
          <v-icon v-else>more_vert</v-icon>
        </v-btn>
      </template>
      <v-tooltip v-for="(item, index) in items" left :key="index">
          <template v-slot:activator="tooltip">
              <v-btn fab small v-on="tooltip.on" @click="action(item)">
                <v-icon small>{{ item.icon }}</v-icon>
              </v-btn>
          </template>
          {{ item.tooltip }}
      </v-tooltip>
    </v-speed-dial>
    """
    def vue_action(self, data):
        print(data)
        name = data['value']
        self.backend.figure.interaction = self.backend.tool_actions_map[name]
items = [
    {'value': 'pan/zoom', 'icon': 'pan_tool', 'tooltip': "Pan & zoom"},
    {'value': 'select', 'icon':'crop_free', 'tooltip': "Square selection"}, 
]
action = Action(items=items, backend=plot_sky.backend)
v.Html(tag='div', pa_10=True, style_='width: 400px; height: 400px; position: relative', children=[action])

In [ ]:
class MainContent(v.VuetifyTemplate):
    loading = traitlets.Bool(False).tag(sync=True)
    @traitlets.default('template')
    def _template(self):
        return f"""
    <v-container class="grey lighten-5">
        <v-row justify="center">
            <v-col xs="12" md="6" lg="4" xl="3">
                <v-card style="height: 100%" :loading="loading"
                    class="mx-auto"
                  >
                    <v-card-title style="padding-left: 50px; height=65px">Overview</v-card-title>
                    <v-card-text  style="padding-left: 50px">
                        This <a href="https://github.com/QuantStack/voila" target="_blank">voila</a> dashboard
                        show the <a href="https://gea.esac.esa.int/archive/" target="_blank">Gaia</a> dataset, crossmatched
                        with the <a href="https://panstarrs.stsci.edu/" target="_blank">Pan-STARRS</a> dataset. You are looking at:
                        <p>
                            <ul>
                                <li>{len(df):,} stars.</li>
                                <li><counter/></li>
                                <li><counter_selection/></li>
                            </ul>
                        </p>
                        <p>
                            The visualizations are calculated on the fly using <a href="https://github.com/vaexio/vaex/" target="_blank">vaex</a>
                            to process the nearly 1 billion stars interactively.
                        </p>
                        <p>
                            The visualizations on the rights and bottom can be used for filter the data visually.
                            Apart from a visual selection, this following expression is included for filtering as well.
                        </p>
                            <filter-editor/>
                        
                        
                    </v-card-text>
                </v-card>
            </v-col>
            <v-col xs="12" md="6" lg="4" xl="3">
                <v-card  :loading="loading" style="height: 100%" 
                    class="mx-auto"
                  >
                    <v-card-title style="padding-left: 50px;">Sky view</v-card-title>
                    <v-card-text  style="padding-left: 50px">Navigate the sky or select a region of the sky to filter the HR diagram.</v-card-text>
                    <sky/>
                </v-card>
            </v-col>
            <v-col xs="12" md="6" lg="4" xl="3">
                <v-card  style="height: 100%"  :loading="loading"
                    class="mx-auto"
                  >
                    <v-card-title style="padding-left: 50px;">Magnitude distribution</v-card-title>
                    <v-card-text  style="padding-left: 50px">Histogram of the Gaia G manitudes. Select a region to filter the HR diagram.</v-card-text>
                    <g-mag/>
                </v-card>
            </v-col>
            <v-col xs="12" md="6" lg="4" xl="3">
                <v-card  :loading="loading" style="height: 100%" 
                    class="mx-auto"
                  >
                    <v-card-title style="padding-left: 50px;">HR diagram</v-card-title>
                    <v-card-text  style="padding-left: 50px">
                        Shows a Hertzsprung Russell diagram of the sky selection, the magnitude selection and the custom
                        filter combined.
                    </v-card-text>
                    <hr-diagram/>
                </v-card>
            </v-col>
        </v-row>
    </v-container>
"""
    
widget_main = MainContent(components={
    'sky': v.Html(tag='div', style_='position: relative', children=[plot_sky.backend.widget, action]),
    'hr-diagram': widgets.VBox([plot_hrd.backend.widget]),
    'g-mag': fig1d,
    'counter': counter_processed,
    'counter_selection': counter_selection,
    'filter-editor': widget_selection
})
widget_main

In [ ]:
@df.executor.signal_begin.connect
def progress_begin():
    widget_main.loading = True
@df.executor.signal_end.connect
def progress_update():
    widget_main.loading = False


In [ ]:
import traitlets
class Main(v.VuetifyTemplate):
    previous = traitlets.List(traitlets.Unicode()).tag(sync=True)
    current = traitlets.List(traitlets.Unicode()).tag(sync=True)
    firsts = traitlets.List(traitlets.Bool(True)).tag(sync=True)
    value = traitlets.Integer()
    format = traitlets.Unicode('{: 14,d}')
    postfix = traitlets.Unicode('').tag(sync=True)
    title = traitlets.Unicode('Vaex').tag(sync=True)
    components = traitlets.Dict(None, allow_none=True).tag(sync=True, **widgets.widget.widget_serialization)
    items = traitlets.List().tag(sync=True)
    showNavBar = traitlets.Bool(False).tag(sync=True)
    
    @traitlets.observe('value')
    def _value(self, change):
        text = self.format.format(self.value)
        self.previous = self.current
        self.current = [k.replace(' ', '&nbsp;') for k in text]
        if self.previous is None:
            self.current = self.previous

    template = traitlets.Unicode(f'''
        <v-layout>
            <v-navigation-drawer v-model="showNavBar" absolute app temporary style="width: 400px">
                
                <content-nav/>
            </v-navigation-drawer>
            <v-app-bar app absolute>
                <v-app-bar-nav-icon
                        @click.stop="showNavBar = !showNavBar"
                ></v-app-bar-nav-icon>
                <v-toolbar-title>
                    <h2 style="color: #666">
                    <img style="height: 30px" src="https://vaex.io/img/logos/logo.svg"></img>
                    Gaia Discovery Engine with <a href="https://github.com/vaexio/vaex/" target="_blank">vaex</a> - 1 billion stars
                    </h2>
                </v-toolbar-title>
                <div class="flex-grow-1"></div>
                <progress_circular/>
            </v-app-bar>

            <v-content>
                <content-main/>
            </v-content>
        </v-layout>
      ''').tag(sync=True)

main = Main(items=[{'title': 'Bla'}],
           components={
               'content-main': widget_main,
               'content-nav': column_list,
               'content-progress': progress,
               'content-filter': widget_selection,
               'progress_circular': progress_circular
           })
main

In [ ]:
v.Layout( _metadata={'mount_id': 'content'}, children=[main]);

In [ ]:
progress_circular

In [ ]:
fig1d.background_style = {'fill': 'none'}

In [ ]:
plot_hrd.backend.figure.background_style = {'fill': 'none'}
plot_sky.backend.figure.background_style = {'fill': 'none'}

In [ ]:
plot_hrd.backend.figure.axes[1].grid_lines = 'none'
plot_sky.backend.figure.axes[1].grid_lines = 'none'

In [ ]:
progress_circular.value = 80

In [ ]:
progress_circular.size = 30
progress_circular.width = 5

In [ ]:
progress_circular.color = '#666'

In [ ]:
widget_main.loading = False